In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### COLAB DRIVE

---



In [ ]:
#@title RESET AND FILES NAME
%reset -f
#path data export
model_folder = "model_test/"

# list your files in the exact order you want their rows to appear
file_names = [
    "Aida_Export_TESTin_0_1000.xlsx",
    "Aida_Export_TESTout_100001_101000.xlsx",
]

test_files = [
    "Aida_Export_testD_001_200.xlsx",
    "Aida_Export_testD_201_400.xlsx",
    "Aida_Export_testD_401_600.xlsx",
    "Aida_Export_testD_601_800.xlsx",
    "Aida_Export_testD_801_1000.xlsx",
    "Aida_Export_testD_100001_100100.xlsx",
    "Aida_Export_testD_100101_100200.xlsx",
    "Aida_Export_testD_100201_100400.xlsx",
    "Aida_Export_testD_100401_100600.xlsx",
    "Aida_Export_testD_100601_100800.xlsx",
    "Aida_Export_testD_100801_101000.xlsx"
]

In [ ]:
path_data = "/content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/DATA/" + model_folder
path_MODEL = "/content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/output/" + model_folder

In [ ]:
from IPython.display import clear_output
!pip install pandas
#!pip install scikit-learn
#!pip install optuna

import pandas as pd
import os
import numpy as np
import re

"""
def safe_literal_eval(x):
    # If x is a list or numpy array and empty, return an empty list
    if isinstance(x, (list, np.ndarray)) and len(x) == 0:
        return []
    # For scalar values, check if x is NaN
    if not isinstance(x, (list, np.ndarray)) and pd.isna(x):
        return []

    # Otherwise, convert to string and strip any whitespace
    x = str(x).strip()

    try:
        return ast.literal_eval(x)
    except Exception:
        return x
"""
clear_output()

### DATA MANIPULATION

---




In [ ]:
#@title FILES ≥ 15 (with per–chunk date/year fixing)

# 1) Regex for “year” or “date” at end of column name
date_pattern = re.compile(r'(year|date)$', re.IGNORECASE)

df_list = []
for fn in file_names:               # assumes file_names & path_data exist
    print(f"Processing file: {fn}")
    tmp = pd.read_excel(
        path_data + fn,
        sheet_name="Results",
        index_col=None,
        na_values=["n.a."],       # if you still have literal "n.a."
        keep_default_na=True
    )
    # 2) Normalize your key
    tmp['Tax code number'] = tmp['Tax code number'].astype('Int64')

    # 3) Find & fix every date/year column in this chunk
    for col in tmp.columns:
        if not date_pattern.search(col):
            continue

        ser = tmp[col]
        # a) Excel‐serial branch (for floats/ints)
        ser_num = pd.to_numeric(ser, errors='coerce')
        dt = pd.to_datetime(
            ser_num,
            unit='D',
            origin='1899-12-30',
            errors='coerce'
        )

        # b) String‐date fallback
        mask = dt.isna() & ser.notna()
        if mask.any():
            dt.loc[mask] = pd.to_datetime(
                ser[mask],
                dayfirst=True,
                errors='coerce'
            )

        tmp[col] = dt
        # debug print
        total = ser.notna().sum()
        good  = dt.notna().sum()
        print(f"  {col}: parsed {good}/{total} values into dates")

    df_list.append(tmp)

# 4) Concatenate end‐to‐end
df_start = pd.concat(df_list, ignore_index=True)
print("Final shape:", df_start.shape)


#df.iloc[46847:46857]
#df.tail(1300)
df_start.shape
#df_start.iloc[3000:3050]
#df_start

Processing file: Aida_Export_TESTin_0_1000.xlsx
  Incorporation year: parsed 998/998 values into dates
  Last accounting closing date: parsed 1000/1000 values into dates
Processing file: Aida_Export_TESTout_100001_101000.xlsx
  Incorporation year: parsed 1000/1000 values into dates
  Last accounting closing date: parsed 1000/1000 values into dates
Final shape: (3273, 16)


(3273, 16)

In [ ]:
#@title Events Manipulation

# 2) Build your 'master' enterprises (one row per tax code)
master = (
    df_start[df_start['Tax code number'].notna()]
      .drop_duplicates(subset='Tax code number', keep='first')
)

# 3) Prepare the events table
events = (
    df_start[['Tax code number',
        'Procedure/cessazione',
        'Date of open procedure/cessazione',
        'Date of closure procedure']]
    .copy()
)
# fill down the tax‐code so every event row has its enterprise key
events['Tax code number'] = events['Tax code number'].ffill()

# keep only real events
events = events[events['Procedure/cessazione'].notna()].rename(columns={
    'Procedure/cessazione':        'Procedure',
    'Date of open procedure/cessazione': 'open_date',
    'Date of closure procedure':   'close_date'
})

# parse dates
events['open_date']  = pd.to_datetime(events['open_date'],  dayfirst=True, errors='coerce')
events['close_date'] = pd.to_datetime(events['close_date'], dayfirst=True, errors='coerce')

# 4) Sort & number each event per enterprise
events = (events
          .sort_values(['Tax code number', 'open_date'], ascending=[True, False]))
events['event_idx'] = events.groupby('Tax code number').cumcount()
events['event_idx'] = -events['event_idx']

# 5) Pivot to wide with unstack
df_e = events.set_index(['Tax code number','event_idx'])[['Procedure','open_date','close_date']]
df_un = df_e.unstack('event_idx')  # shape: index=tax codes, columns=(field, idx)
#5.1 order of tuples
field_order = ['Procedure','open_date','close_date']
idx_order   = sorted(df_un.columns.levels[1], reverse=True)

ordered_tuples = [
    (field, idx)
    for field in field_order
    for idx   in idx_order
    if (field, idx) in df_un.columns
]

df_un = df_un.reindex(columns=pd.MultiIndex.from_tuples(ordered_tuples))
# 6) Flatten the MultiIndex columns
df_un.columns = [
    (f"Event {i}"                 if field=='Procedure' else
     f"Date of open Event {i}"   if field=='open_date' else
     f"Date of closure Event {i}")
    for field, i in df_un.columns
]
df_un = df_un.reset_index()  # bring back Tax code number as a column

# 7) Merge back onto master (preserves your enterprise order)
final_df = (master
            .merge(df_un, on='Tax code number', how='left', sort=False)
            .reset_index(drop=True)
           )

#print(f"Enterprises: {len(master)}, rows in final_df: {len(final_df)}")
#final_df.head()

final_df = final_df.drop(columns=[
    'Procedure/cessazione',
    'Date of open procedure/cessazione',
    'Date of closure procedure'
])

In [ ]:
#@title Appending
# 1) Treat only "n.a." as missing when reading
read_kwargs = dict(
    sheet_name="Results",
    index_col=None,
    na_values=["n.a."],
    keep_default_na=True
)

# 2) Read & stack all the new data end-to-end
df_list = []
for fn in test_files:
    tmp = pd.read_excel(path_data + fn, **read_kwargs)
    tmp['Tax code number'] = tmp['Tax code number'].astype('Int64')
    df_list.append(tmp)

In [ ]:
#@title Merging

# 1) Stack all new files into one DataFrame
all_new = pd.concat(df_list, ignore_index=True)

# 2) Ensure the key is the same type
all_new['Tax code number'] = all_new['Tax code number'].astype('Int64')

# 3) Perform a 1:1 left-join of your master with the new data,
df = final_df.merge(
    all_new,
    on='Tax code number',
    how='left',
    validate='one_to_one'
)

print(df.shape)  # rows == final_df.rows; columns == union of both sets of variables


(2000, 4276)


In [ ]:
#@title DATA MANIPULATION

In [ ]:
#@title visualize big database

# Show *every* column
pd.set_option('display.max_columns', 1000)

# (Optionally widen your display so lines don’t wrap too crazily)
pd.set_option('display.width', 20)
pd.set_option('display.max_colwidth', 50)

# Now when you do…
#df

#print(final_df.iloc[:, :10])
#df.iloc[:30, :200]
#df.iloc[1000:1030, :200]

df.iloc[985:1015, :200]

,Unnamed: 0_x,Company name_x,Tax code number,Registered office address - Postal code,Registered office address - Longitude,Registered office address - Latitude,Trading address - Postal code,Legal status,Legal form,Company category,Incorporation year,No of available years,Last accounting closing date,Event 0,Event -1,Event -2,Event -3,Event -4,Event -5,Event -6,Date of open Event 0,Date of open Event -1,Date of open Event -2,Date of open Event -3,Date of open Event -4,Date of open Event -5,Date of open Event -6,Date of closure Event 0,Date of closure Event -1,Date of closure Event -2,Date of closure Event -3,Date of closure Event -4,Date of closure Event -5,Date of closure Event -6,Unnamed: 0_y,Company name_y,Revenues from sales and services\nth EUR\n2023,Revenues from sales and services\nth EUR\n2022,Revenues from sales and services\nth EUR\n2021,Revenues from sales and services\nth EUR\n2020,Revenues from sales and services\nth EUR\n2019,Revenues from sales and services\nth EUR\n2018,Revenues from sales and services\nth EUR\n2017,Revenues from sales and services\nth EUR\n2016,Revenues from sales and services\nth EUR\n2015,Revenues from sales and services\nth EUR\n2014,EBITDA\nth EUR\n2023,EBITDA\nth EUR\n2022,EBITDA\nth EUR\n2021,EBITDA\nth EUR\n2020,EBITDA\nth EUR\n2019,EBITDA\nth EUR\n2018,EBITDA\nth EUR\n2017,EBITDA\nth EUR\n2016,EBITDA\nth EUR\n2015,EBITDA\nth EUR\n2014,Profit (loss)\nth EUR\n2023,Profit (loss)\nth EUR\n2022,Profit (loss)\nth EUR\n2021,Profit (loss)\nth EUR\n2020,Profit (loss)\nth EUR\n2019,Profit (loss)\nth EUR\n2018,Profit (loss)\nth EUR\n2017,Profit (loss)\nth EUR\n2016,Profit (loss)\nth EUR\n2015,Profit (loss)\nth EUR\n2014,Total assets\nth EUR\n2023,Total assets\nth EUR\n2022,Total assets\nth EUR\n2021,Total assets\nth EUR\n2020,Total assets\nth EUR\n2019,Total assets\nth EUR\n2018,Total assets\nth EUR\n2017,Total assets\nth EUR\n2016,Total assets\nth EUR\n2015,Total assets\nth EUR\n2014,Total shareholder's funds\nth EUR\n2023,Total shareholder's funds\nth EUR\n2022,Total shareholder's funds\nth EUR\n2021,Total shareholder's funds\nth EUR\n2020,Total shareholder's funds\nth EUR\n2019,Total shareholder's funds\nth EUR\n2018,Total shareholder's funds\nth EUR\n2017,Total shareholder's funds\nth EUR\n2016,Total shareholder's funds\nth EUR\n2015,Total shareholder's funds\nth EUR\n2014,Net financial position\nth EUR\n2023,Net financial position\nth EUR\n2022,Net financial position\nth EUR\n2021,Net financial position\nth EUR\n2020,Net financial position\nth EUR\n2019,Net financial position\nth EUR\n2018,Net financial position\nth EUR\n2017,Net financial position\nth EUR\n2016,Net financial position\nth EUR\n2015,Net financial position\nth EUR\n2014,EBITDA/Vendite\n%\n2023,EBITDA/Vendite\n%\n2022,EBITDA/Vendite\n%\n2021,EBITDA/Vendite\n%\n2020,EBITDA/Vendite\n%\n2019,EBITDA/Vendite\n%\n2018,EBITDA/Vendite\n%\n2017,EBITDA/Vendite\n%\n2016,EBITDA/Vendite\n%\n2015,EBITDA/Vendite\n%\n2014,Return on sales (ROS)\n%\n2023,Return on sales (ROS)\n%\n2022,Return on sales (ROS)\n%\n2021,Return on sales (ROS)\n%\n2020,Return on sales (ROS)\n%\n2019,Return on sales (ROS)\n%\n2018,Return on sales (ROS)\n%\n2017,Return on sales (ROS)\n%\n2016,Return on sales (ROS)\n%\n2015,Return on sales (ROS)\n%\n2014,Return on asset (ROA)\n%\n2023,Return on asset (ROA)\n%\n2022,Return on asset (ROA)\n%\n2021,Return on asset (ROA)\n%\n2020,Return on asset (ROA)\n%\n2019,Return on asset (ROA)\n%\n2018,Return on asset (ROA)\n%\n2017,Return on asset (ROA)\n%\n2016,Return on asset (ROA)\n%\n2015,Return on asset (ROA)\n%\n2014,Return on equity (ROE)\n%\n2023,Return on equity (ROE)\n%\n2022,Return on equity (ROE)\n%\n2021,Return on equity (ROE)\n%\n2020,Return on equity (ROE)\n%\n2019,Return on equity (ROE)\n%\n2018,Return on equity (ROE)\n%\n2017,Return on equity (ROE)\n%\n2016,Return on equity (ROE)\n%\n2015,Return on equity (ROE)\n%\n2014,Debt/equity ratio\n%\n2023,Debt/equity ratio\n%\n2022,Debt/equity ratio\n%\n2021,Debt/equity ratio\n%\

In [ ]:
print(df.shape)

(2000, 4276)


In [ ]:
#@title visualize big database

# Show *every* column
pd.set_option('display.max_columns', 1000)

# (Optionally widen your display so lines don’t wrap too crazily)
pd.set_option('display.width', 20)
pd.set_option('display.max_colwidth', 50)

# Now when you do…

#print(final_df.iloc[:, :10])
df.iloc[985:1015, :200]

,Unnamed: 0_x,Company name_x,Tax code number,Registered office address - Postal code,Registered office address - Longitude,Registered office address - Latitude,Trading address - Postal code,Legal status,Legal form,Company category,Incorporation year,No of available years,Last accounting closing date,Event 0,Event -1,Event -2,Event -3,Event -4,Event -5,Event -6,Date of open Event 0,Date of open Event -1,Date of open Event -2,Date of open Event -3,Date of open Event -4,Date of open Event -5,Date of open Event -6,Date of closure Event 0,Date of closure Event -1,Date of closure Event -2,Date of closure Event -3,Date of closure Event -4,Date of closure Event -5,Date of closure Event -6,Unnamed: 0_y,Company name_y,Revenues from sales and services\nth EUR\n2023,Revenues from sales and services\nth EUR\n2022,Revenues from sales and services\nth EUR\n2021,Revenues from sales and services\nth EUR\n2020,Revenues from sales and services\nth EUR\n2019,Revenues from sales and services\nth EUR\n2018,Revenues from sales and services\nth EUR\n2017,Revenues from sales and services\nth EUR\n2016,Revenues from sales and services\nth EUR\n2015,Revenues from sales and services\nth EUR\n2014,EBITDA\nth EUR\n2023,EBITDA\nth EUR\n2022,EBITDA\nth EUR\n2021,EBITDA\nth EUR\n2020,EBITDA\nth EUR\n2019,EBITDA\nth EUR\n2018,EBITDA\nth EUR\n2017,EBITDA\nth EUR\n2016,EBITDA\nth EUR\n2015,EBITDA\nth EUR\n2014,Profit (loss)\nth EUR\n2023,Profit (loss)\nth EUR\n2022,Profit (loss)\nth EUR\n2021,Profit (loss)\nth EUR\n2020,Profit (loss)\nth EUR\n2019,Profit (loss)\nth EUR\n2018,Profit (loss)\nth EUR\n2017,Profit (loss)\nth EUR\n2016,Profit (loss)\nth EUR\n2015,Profit (loss)\nth EUR\n2014,Total assets\nth EUR\n2023,Total assets\nth EUR\n2022,Total assets\nth EUR\n2021,Total assets\nth EUR\n2020,Total assets\nth EUR\n2019,Total assets\nth EUR\n2018,Total assets\nth EUR\n2017,Total assets\nth EUR\n2016,Total assets\nth EUR\n2015,Total assets\nth EUR\n2014,Total shareholder's funds\nth EUR\n2023,Total shareholder's funds\nth EUR\n2022,Total shareholder's funds\nth EUR\n2021,Total shareholder's funds\nth EUR\n2020,Total shareholder's funds\nth EUR\n2019,Total shareholder's funds\nth EUR\n2018,Total shareholder's funds\nth EUR\n2017,Total shareholder's funds\nth EUR\n2016,Total shareholder's funds\nth EUR\n2015,Total shareholder's funds\nth EUR\n2014,Net financial position\nth EUR\n2023,Net financial position\nth EUR\n2022,Net financial position\nth EUR\n2021,Net financial position\nth EUR\n2020,Net financial position\nth EUR\n2019,Net financial position\nth EUR\n2018,Net financial position\nth EUR\n2017,Net financial position\nth EUR\n2016,Net financial position\nth EUR\n2015,Net financial position\nth EUR\n2014,EBITDA/Vendite\n%\n2023,EBITDA/Vendite\n%\n2022,EBITDA/Vendite\n%\n2021,EBITDA/Vendite\n%\n2020,EBITDA/Vendite\n%\n2019,EBITDA/Vendite\n%\n2018,EBITDA/Vendite\n%\n2017,EBITDA/Vendite\n%\n2016,EBITDA/Vendite\n%\n2015,EBITDA/Vendite\n%\n2014,Return on sales (ROS)\n%\n2023,Return on sales (ROS)\n%\n2022,Return on sales (ROS)\n%\n2021,Return on sales (ROS)\n%\n2020,Return on sales (ROS)\n%\n2019,Return on sales (ROS)\n%\n2018,Return on sales (ROS)\n%\n2017,Return on sales (ROS)\n%\n2016,Return on sales (ROS)\n%\n2015,Return on sales (ROS)\n%\n2014,Return on asset (ROA)\n%\n2023,Return on asset (ROA)\n%\n2022,Return on asset (ROA)\n%\n2021,Return on asset (ROA)\n%\n2020,Return on asset (ROA)\n%\n2019,Return on asset (ROA)\n%\n2018,Return on asset (ROA)\n%\n2017,Return on asset (ROA)\n%\n2016,Return on asset (ROA)\n%\n2015,Return on asset (ROA)\n%\n2014,Return on equity (ROE)\n%\n2023,Return on equity (ROE)\n%\n2022,Return on equity (ROE)\n%\n2021,Return on equity (ROE)\n%\n2020,Return on equity (ROE)\n%\n2019,Return on equity (ROE)\n%\n2018,Return on equity (ROE)\n%\n2017,Return on equity (ROE)\n%\n2016,Return on equity (ROE)\n%\n2015,Return on equity (ROE)\n%\n2014,Debt/equity ratio\n%\n2023,Debt/equity ratio\n%\n2022,Debt/equity ratio\n%\n2021,Debt/equity ratio\n%\

In [ ]:
#@title LIST MISSING VARIABLES

# 1) Count columns that are 100% missing
all_nan = df.columns[df.isna().all()]
print(f"Columns with all NaN: {len(all_nan)}")

# 2) Drop them
df.drop(columns=all_nan, inplace=True)

# 3) Show the list of dropped columns
print("Dropped columns:")
for col in all_nan:
    print(f" - {col}")


In [ ]:
#@title RANK OF MISSING in NON-MISSING VARIABLES

# 4) Compute missing counts for the rest, sort descending
missing_counts = df.isna().sum().sort_values(ascending=False)

# Ensure pandas will print all rows
pd.set_option('display.max_rows', None)

print("\nMissing‐value counts for remaining columns (desc):")
print(missing_counts)


In [ ]:
#@title SAVE PROCESSED DATASET
import pickle

# Save the processed dataset
save_path = path_data + 'processed_df_start.pkl'
df_start.to_pickle(save_path)
print(f"✅ Saved processed dataset to: {save_path}")
print(f"Shape: {df_start.shape}, Size: {df_start.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

✅ Saved processed dataset to: /content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/DATA/model_test/processed_df_start.pkl
Shape: (3273, 16), Size: 1.5 MB


### (#) open dataset created

In [ ]:
#@title LOAD PROCESSED DATASET (with missing) BIG PROBLEM
import pickle

# Load the pre-processed dataset
load_path = path_data + 'processed_df_start.pkl'

try:
    df_start = pd.read_pickle(load_path)
    print(f"✅ Loaded processed dataset from: {load_path}")
    print(f"Shape: {df_start.shape}")
    print(f"Columns with 'date'/'year': {len([c for c in df_start.columns if re.search(r'(year|date)$', c, re.IGNORECASE)])}")

    # Quick validation that dates are still datetime objects
    date_cols = [c for c in df_start.columns if re.search(r'(year|date)$', c, re.IGNORECASE)]
    if date_cols:
        sample_col = date_cols[0]
        print(f"Date dtype check - {sample_col}: {df_start[sample_col].dtype}")

except FileNotFoundError:
    print("❌ Processed file not found. Run the full import process first.")
except Exception as e:
    print(f"❌ Error loading processed file: {e}")

✅ Loaded processed dataset from: /content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/DATA/model_test/processed_df_start.pkl
Shape: (3273, 16)
Columns with 'date'/'year': 2
Date dtype check - Incorporation year: datetime64[ns]


### stats

---



In [ ]:
#@title missing variables
# 1) Row‐count sanity: you should still have exactly one row per tax code
n_rows = final_df.shape[0]
n_unique_codes = final_df['Tax code number'].nunique()
print(f"Rows: {n_rows}, Unique Tax codes: {n_unique_codes}")

# 2) Check for any duplicate tax codes (should be zero)
dups = final_df['Tax code number'].duplicated().sum()
print(f"Duplicate keys: {dups}")

# 3) Overall missing‐value summary
missing_per_col = final_df.isna().sum().sort_values(ascending=False)
print("Top 10 columns by # missing values:")
print(missing_per_col.iloc[30:90])

Rows: 2000, Unique Tax codes: 2000
Duplicate keys: 0
Top 10 columns by # missing values:
Tax code number                            0
Legal status                               0
Company category                           0
Registered office address - Postal code    0
dtype: int64


In [ ]:
#@title only missing variables

# 1) Compute total missing per column
missing = final_df.isna().sum()

# 2) Filter to only those with at least one missing
missing = missing[missing > 0]

# 3) Sort descending (largest gaps first)
missing = missing.sort_values(ascending=False)

# 4) Make sure pandas will print every row
pd.set_option('display.max_rows', None)

# 5) Now display
print(missing)


Total cash and cash equivalents at year end\n2018    2000
Total cash and cash equivalents at year end\n2020    2000
Total cash and cash equivalents at year end\n2019    2000
Total cash and cash equivalents at year end\n2021    2000
Total cash and cash equivalents at year end\n2022    2000
Total cash and cash equivalents at year end\n2023    2000
Date of closure Event -6                             2000
Total cash and cash equivalents at year end\n2017    2000
Total cash and cash equivalents at year end\n2014    2000
Total cash and cash equivalents at year end\n2016    2000
Total cash and cash equivalents at year end\n2015    2000
Date of closure Event -5                             1998
Event -6                                             1998
Date of open Event -6                                1998
Date of closure Event -4                             1997
Event -5                                             1995
Date of open Event -5                                1995
Date of closur

In [ ]:
#@title visualize big part of database

from google.colab import data_table

# Turn on Colab’s interactive DataFrame display
data_table.enable_dataframe_formatter()

# Now this will show you an interactive, scrollable table
# (you can still slice it to a column range if you like)
#display(final_df)

# — or to only see columns 20 through 39 —
display(final_df.iloc[:, 170:200])


In [ ]:
#@title CHECK COLLINEARITY, MISSINGNESS & MORE (Corrected)

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 0) Use your final, one-row-per-enterprise DataFrame
df = final_df.copy()

# 1) Missing‐value summary on final_df
miss_pct = df.isnull().mean().sort_values(ascending=False)
print(">> Top 10 columns by percent missing:")
print(miss_pct.head(100).apply(lambda x: f"{x:.1%}"))
print("\n>> Columns with 100% missing:")
print(miss_pct[miss_pct == 1.0].index.tolist())

# 2) Data‐type & cardinality overview (drop the list‐column if present)
if 'ALL_events' in df.columns:
    basic = df.drop(columns=['ALL_events'])
else:
    basic = df.copy()

nunique = basic.nunique(dropna=False)
dtype_df = pd.DataFrame({
    'dtype':    basic.dtypes.astype(str),
    'n_unique': nunique
})
print("\n>> Data types and unique counts (sorted by dtype):")
print(dtype_df.sort_values('n_unique').head(10))

# 3) Correlation on numeric features (excluding fully‐missing columns)
#    Select columns with <100% missing, then pick numeric
valid_cols = miss_pct[miss_pct < 1.0].index
num = df[valid_cols].select_dtypes(include=[np.number])

corr = num.corr().abs()
high_corr = (
    corr.where(np.tril(np.ones(corr.shape), k=-1).astype(bool))
        .stack()
        .loc[lambda s: s > 0.90]
        .sort_values(ascending=False)
)
print(f"\n>> Highly correlated pairs (>|0.90|) [count={len(high_corr)}]:")
print(high_corr.head(10))

# 4) Variance Inflation Factor (VIF)
num_for_vif = num.fillna(0).astype(float)
vif_df = pd.DataFrame({
    'variable': num_for_vif.columns,
    'VIF': [
        variance_inflation_factor(num_for_vif.values, i)
        for i in range(num_for_vif.shape[1])
    ]
}).sort_values('VIF', ascending=False)
print("\n>> Top 10 variables by VIF:")
print(vif_df.head(10))

# 5) Distribution diagnostics for numeric columns
print("\n>> Skewness of numeric features:")
print(num.skew().sort_values(ascending=False).head(10))

outliers = ((num - num.mean()).abs() > 3 * num.std()) \
            .sum() \
            .sort_values(ascending=False)
print("\n>> Count of outliers (>|3σ|) per numeric feature:")
print(outliers.head(10))


### dummies bunkruptcy

In [ ]:
final_df

In [ ]:
#@title Create All list
event_cols = [c for c in final_df.columns if c.startswith('Event ')]
final_df['ALL_events'] = final_df[event_cols] \
    .apply(lambda row: [e for e in row.dropna().tolist()], axis=1)


In [ ]:
#@title tabulation of all events in ALL_events

# 1) Explode the ALL_events list into a single long Series
all_events_series = final_df['ALL_events'].explode()

# 2) Compute counts and percentages
counts   = all_events_series.value_counts(dropna=False)
percents = all_events_series.value_counts(normalize=True, dropna=False) * 100

# 3) Assemble into a DataFrame
tab = pd.DataFrame({
    'count':   counts,
    'percent': percents
})
tab.index.name = 'Event'

# 4) Print it (Stata‐style)
print("=== Stata‐style tab of ALL_EVENTS ===")
print(tab.to_string())

tab.to_csv('all_events_tab.csv', sep='|', index=True)
print("Saved pipe-delimited table to all_events_tab.csv")

=== Stata‐style tab of ALL_EVENTS ===
                                                                                                                    count    percent
Event                                                                                                                               
NaN                                                                                                                   844  25.786740
Composition with creditors                                                                                            739  22.578674
Bankruptcy                                                                                                            315   9.624198
Transfer to another province                                                                                          167   5.102353
Winding up and liquidation                                                                                            156   4.766269
Compulsory administrative liqui

In [ ]:
#@title study of BANKRUPTCY
enterprises = len(final_df)

#BANKRUPTCY
final_df['has_bankruptcy']   = final_df['ALL_events'].apply(lambda evs: 'Bankruptcy' in evs).astype(int)

count_bankruptcy   = final_df['has_bankruptcy'].sum()
ratio_bankruptcy   = round(count_bankruptcy   / enterprises * 100, 5)

print(f"Enterprises with ≥1 ‘Bankruptcy’ event: {count_bankruptcy}")
print(f"Total enterprises (rows): {enterprises}")
print(f"Ratio: {ratio_bankruptcy}%\n")

Enterprises with ≥1 ‘Bankruptcy’ event: 301
Total enterprises (rows): 2000
Ratio: 15.05%



In [ ]:
#@title Bankruptcy-LIKE events
# Define non-voluntary event list
nonvol_events = [
    "Bankruptcy",
    "Conclusion of bankruptcy procedures",
    "State of insolvency",
    "Closure due to bankruptcy",
    "Closure due to bankruptcy or liquidatione",
    "Judicial liquidation",
    "Compulsory administrative liquidation",
    "Administrative compulsory liquidation",
    "Court ordered liquidation",
    "Court ordered administration",
    "Extraordinary administration",
    "Extraordinary administration large company",
    "Protective and precautionary measures",
    "Controlled administration"
]

# Identify all Event columns
event_cols = [c for c in final_df.columns if c.startswith('Event ')]

# Create the dummy
final_df['NonVolEvent'] = (
    final_df[event_cols]
      .isin(nonvol_events)
      .any(axis=1)
      .astype(int)
)

# Check distribution
print(final_df['NonVolEvent'].value_counts())

NonVolEvent
0    1423
1     578
Name: count, dtype: int64


In [ ]:
#@title PRE-bankruptcy events
# define your “maybe not voluntary” list
maybe_not_vol = [
    "Debt restructuring agreements",
    "Composition with creditors",
    "Simplified composition with creditors",
    "Creditors agreement",
    "Debt restructuring plan",
    "Post-bankruptcy composition with creditors",
    "Restructuring plan subject to validation/approval"
]

# create the dummy: 1 if any Event N is in that list
final_df['MaybeNonVolEvent'] = (
    final_df[event_cols]
      .isin(maybe_not_vol)
      .any(axis=1)
      .astype(int)
)

# check its distribution
print(final_df['MaybeNonVolEvent'].value_counts())

MaybeNonVolEvent
0    1526
1     475
Name: count, dtype: int64


In [ ]:
#@title all togheder

# Combined dummy: 1 if NonVolEvent==1 OR MaybeNonVolEvent==1
final_df['SeriousEvent'] = (
    (final_df['NonVolEvent'] == 1) |
    (final_df['MaybeNonVolEvent'] == 1)
).astype(int)

# Quick check
print(final_df['SeriousEvent'].value_counts())

SeriousEvent
0    1011
1     990
Name: count, dtype: int64


### TEST COLAB

---



In [ ]:
#@title Reshape Long
import pandas as pd
import numpy as np
import re
import gc

# Starting from `df` which is one row per enterprise

# 1. First, identify columns with years
year_pattern = re.compile(r'(20\d{2})$', re.MULTILINE)
wide_cols = [col for col in df.columns if year_pattern.search(col)]
id_cols = [c for c in df.columns if c not in wide_cols]

print(f"Found {len(wide_cols)} columns with years")
print(f"Number of ID columns: {len(id_cols)}")
print(f"Total enterprises: {len(df)}")

# 2. Extract bankruptcy years BEFORE reshaping
bankruptcy_years = {}
event_cols = [c for c in df.columns if c.startswith('Event ') and not c.startswith('Date of')]

for idx, row in df.iterrows():
    tax_code = row['Tax code number']
    bankruptcy_year = None

    for event_col in event_cols:
        event_value = row[event_col]
        if pd.notna(event_value) and 'bankruptcy' in str(event_value).lower():
            event_num = event_col.replace('Event ', '')
            date_col = f'Date of open Event {event_num}'
            if date_col in df.columns and pd.notna(row[date_col]):
                try:
                    bankruptcy_date = pd.to_datetime(row[date_col])
                    if pd.notna(bankruptcy_date):
                        bankruptcy_year = bankruptcy_date.year
                        break
                except:
                    pass

    if bankruptcy_year:
        bankruptcy_years[tax_code] = bankruptcy_year

print(f"\nFound {len(bankruptcy_years)} enterprises with bankruptcy")

# 3. Create a function to extract variable name and year
def parse_column_name(col_name):
    """Extract clean variable name and year from column"""
    year_match = year_pattern.search(col_name)
    if year_match:
        year = int(year_match.group(1))
        # Get variable name (first line before \n)
        var_name = col_name.split('\n')[0].strip()
        return var_name, year
    return None, None

# 4. Get unique years and variables
years = set()
variables = set()
col_mapping = {}  # Maps (variable, year) to original column name

for col in wide_cols:
    var_name, year = parse_column_name(col)
    if var_name and year:
        years.add(year)
        variables.add(var_name)
        col_mapping[(var_name, year)] = col

years = sorted(years)
variables = sorted(variables)

print(f"\nYears found: {min(years)} to {max(years)}")
print(f"Number of unique variables: {len(variables)}")
print(f"Expected long format rows: {len(df) * len(years):,}")

# 5. Process in chunks to save memory
chunk_size = 100  # Process 100 enterprises at a time
long_chunks = []

print("\nProcessing in chunks to save memory...")

for chunk_start in range(0, len(df), chunk_size):
    chunk_end = min(chunk_start + chunk_size, len(df))

    # Get chunk of enterprises
    df_chunk = df.iloc[chunk_start:chunk_end]

    # Create list to store long format data for this chunk
    chunk_data = []

    for _, row in df_chunk.iterrows():
        tax_code = row['Tax code number']
        bankrupt_year = bankruptcy_years.get(tax_code, None)

        # For each year
        for year in years:
            # Skip years after bankruptcy
            if bankrupt_year and year > bankrupt_year:
                continue

            # Create row for this enterprise-year
            long_row = {}

            # Add ID columns
            for id_col in id_cols:
                long_row[id_col] = row[id_col]

            # Add year
            long_row['Year'] = year

            # Add variable values for this year
            for var_name in variables:
                col_key = (var_name, year)
                if col_key in col_mapping:
                    orig_col = col_mapping[col_key]
                    if orig_col in row.index:
                        long_row[var_name] = row[orig_col]
                    else:
                        long_row[var_name] = np.nan
                else:
                    long_row[var_name] = np.nan

            chunk_data.append(long_row)

    # Convert chunk to dataframe
    chunk_long = pd.DataFrame(chunk_data)
    long_chunks.append(chunk_long)

    # Progress update
    if (chunk_end % 500 == 0) or (chunk_end == len(df)):
        print(f"  Processed {chunk_end}/{len(df)} enterprises...")

    # Clean up memory
    del chunk_data
    gc.collect()

# 6. Combine all chunks
print("\nCombining chunks...")
long_df = pd.concat(long_chunks, ignore_index=True)

# Clean up
del long_chunks
gc.collect()

# 7. Add derived columns
print("\nAdding derived columns...")

# Find name column
name_col = next((c for c in id_cols if 'name' in c.lower() and c != 'Tax code number'), 'Company name')
if name_col in long_df.columns:
    long_df['NameYear'] = long_df[name_col].astype(str) + ' ' + long_df['Year'].astype(str)
    print(f"Using '{name_col}' as name column")

# Add KeyYear
long_df['KeyYear'] = long_df['Tax code number'].astype(str) + ' ' + long_df['Year'].astype(str)

# Add bankruptcy year
long_df['bankruptcy_year'] = long_df['Tax code number'].map(bankruptcy_years)

# 8. Sort the data
long_df = long_df.sort_values(['Tax code number', 'Year']).reset_index(drop=True)

# 9. Display summary
print(f"\nFinal shape: {long_df.shape}")
print(f"Memory usage: {long_df.memory_usage(deep=True).sum() / 1024**3:.2f} GB")
print(f"Unique enterprises: {long_df['Tax code number'].nunique()}")
print(f"Years covered: {long_df['Year'].min()} to {long_df['Year'].max()}")

# 10. Save to file to avoid losing work
print("\nSaving to pickle file...")
long_df.to_pickle(path_data + 'long_format_data.pkl')
print("Saved successfully!")

# Show sample
print("\nSample of reshaped data:")
display(long_df.head(20))

# Show columns
print("\nColumns in long format (first 30):")
for i, col in enumerate(long_df.columns[:30]):
    print(f"  {i+1}. {col}")

Found 3290 columns with years
Number of ID columns: 227
Total enterprises: 2000

Found 308 enterprises with bankruptcy

Years found: 2014 to 2023
Number of unique variables: 338
Expected long format rows: 20,000

Processing in chunks to save memory...
  Processed 500/2000 enterprises...
  Processed 1000/2000 enterprises...
  Processed 1500/2000 enterprises...
  Processed 2000/2000 enterprises...

Combining chunks...

Adding derived columns...
Using 'Unnamed: 0_x' as name column

Final shape: (19072, 569)
Memory usage: 0.10 GB
Unique enterprises: 1990
Years covered: 2014 to 2023

Saving to pickle file...
Saved successfully!

Sample of reshaped data:


,Unnamed: 0_x,Company name_x,Tax code number,Registered office address - Postal code,Registered office address - Longitude,Registered office address - Latitude,Trading address - Postal code,Legal status,Legal form,Company category,Incorporation year,No of available years,Last accounting closing date,Event 0,Event -1,Event -2,Event -3,Event -4,Event -5,Event -6,Date of open Event 0,Date of open Event -1,Date of open Event -2,Date of open Event -3,Date of open Event -4,Date of open Event -5,Date of open Event -6,Date of closure Event 0,Date of closure Event -1,Date of closure Event -2,Date of closure Event -3,Date of closure Event -4,Date of closure Event -5,Unnamed: 0_y,Company name_y,Additions in progress and advances\nth EUR\n2023.1,Additions in progress and advances\nth EUR\n2022.1,Additions in progress and advances\nth EUR\n2021.1,Additions in progress and advances\nth EUR\n2020.1,Additions in progress and advances\nth EUR\n2019.1,Additions in progress and advances\nth EUR\n2018.1,Additions in progress and advances\nth EUR\n2017.1,Additions in progress and advances\nth EUR\n2016.1,Additions in progress and advances\nth EUR\n2015.1,Additions in progress and advances\nth EUR\n2014.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2023.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2022.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2021.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2020.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2019.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2018.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2017.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2016.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2015.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2014.1,Receiv. due from subs. Comp.\nth EUR\n2023.1,Receiv. due from subs. Comp.\nth EUR\n2022.1,Receiv. due from subs. Comp.\nth EUR\n2021.1,Receiv. due from subs. Comp.\nth EUR\n2020.1,Receiv. due from subs. Comp.\nth EUR\n2019.1,Receiv. due from subs. Comp.\nth EUR\n2018.1,Receiv. due from subs. Comp.\nth EUR\n2017.1,Receiv. due from subs. Comp.\nth EUR\n2016.1,Receiv. due from subs. Comp.\nth EUR\n2015.1,Receiv. due from subs. Comp.\nth EUR\n2014.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2023.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2022.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2021.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2020.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2019.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2018.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2017.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2016.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2015.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2014.2,Due from comp. under parent companies control\nth EUR\n2023.1,Due from comp. under parent companies control\nth EUR\n2022.1,Due from comp. under parent companies control\nth EUR\n2021.1,Due from comp. under parent companies control\nth EUR\n2020.1,Due from comp. under parent companies control\nth EUR\n2019.1,Due from comp. under parent companies control\nth EUR\n2018.1,Due from comp. under parent companies control\nth EUR\n2017.1,Due from comp. under parent companies control\nth EUR\n2016.1,Due from comp. under parent companies control\nth EUR\n2015.1,Due from comp. under parent companies control\nth EUR\n2014.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2023.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2022.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2021.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2020.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2019.1,Due from comp. under p


Columns in long format (first 30):
  1. Unnamed: 0_x
  2. Company name_x
  3. Tax code number
  4. Registered office address - Postal code
  5. Registered office address - Longitude
  6. Registered office address - Latitude
  7. Trading address - Postal code
  8. Legal status
  9. Legal form
  10. Company category
  11. Incorporation year
  12. No of available years
  13. Last accounting closing date
  14. Event 0
  15. Event -1
  16. Event -2
  17. Event -3
  18. Event -4
  19. Event -5
  20. Event -6
  21. Date of open Event 0
  22. Date of open Event -1
  23. Date of open Event -2
  24. Date of open Event -3
  25. Date of open Event -4
  26. Date of open Event -5
  27. Date of open Event -6
  28. Date of closure Event 0
  29. Date of closure Event -1
  30. Date of closure Event -2


In [ ]:
#long_df.head()
#final_df.head()
final_df.iloc[985:1015, :200]
#df.iloc[985:1015, :200]

,Unnamed: 0,Company name,Tax code number,Registered office address - Postal code,Registered office address - Longitude,Registered office address - Latitude,Trading address - Postal code,Legal status,Legal form,Company category,Incorporation year,No of available years,Last accounting closing date,Event 0,Event -1,Event -2,Event -3,Event -4,Event -5,Event -6,Date of open Event 0,Date of open Event -1,Date of open Event -2,Date of open Event -3,Date of open Event -4,Date of open Event -5,Date of open Event -6,Date of closure Event 0,Date of closure Event -1,Date of closure Event -2,Date of closure Event -3,Date of closure Event -4,Date of closure Event -5,Date of closure Event -6
985,986.0,SECI ENERGIA S.P.A.,2602271203,40136.0,11.338145,44.487081,40123.0,Active,Joint stock company - SPA,Limited company,2005-11-23,10.0,2017-12-31,Composition with creditors,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-26,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
986,987.0,COOPERATIVA BASELICE SOCIETA' COOPERATIVA AGRI...,2218090591,4016.0,13.024707,41.300254,NaN,In liquidation,Cooperative company with limited liability by ...,Limited company,2004-07-20,10.0,2017-12-31,Compulsory administrative liquidation,State of insolvency,NaN,NaN,NaN,NaN,NaN,2022-05-10,2021-06-11,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
987,988.0,CO & CI DISTRIBUTORI E DEPOSITI SOCIETA' A RES...,1795590494,57121.0,10.326035,43.561230,57121.0,Active,Limited liability company - SRL,Limited company,2014-10-02,8.0,2022-12-31,Composition with creditors,Protective and precautionary measures,Unified procedure,NaN,NaN,NaN,NaN,2023-09-04,2023-05-10,2023-04-21,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
988,989.0,TINTORIA MORETTI S.R.L.,708470158,20836.0,9.237839,45.701753,20836.0,Active,Limited liability company - SRL,Limited company,1961-04-06,10.0,2023-12-31,Transfer to another province,Composition with creditors,Composition with creditors,Composition with creditors,NaN,NaN,NaN,2013-08-06,2012-08-10,2012-08-10,2012-08-10,NaT,NaT,NaT,NaT,2013-05-23,2018-06-11,NaT,NaT,NaT,NaT
989,990.0,TIPOLITOGRAFIA TRULLO - SOCIETA A RESPONSABILI...,5054751002,148.0,12.438959,41.833578,NaN,Dissolved,Limited liability company - SRL,Limited company,1996-02-22,10.0,2014-12-31,Cancellation from the register of companies,Bankruptcy,Winding up and liquidation,NaN,NaN,NaN,NaN,2020-11-10,2017-06-20,2016-04-28,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
990,991.0,GRADIMARKET SOCIETA' A RESPONSABILITA' LIMITAT...,7447430724,75100.0,16.419107,40.821349,NaN,Active (insolvency proceedings),Limited liability company - SRL,Limited company,2013-02-22,10.0,2023-12-31,Composition with creditors,Protective and precautionary measures,Winding up and liquidation,NaN,NaN,NaN,NaN,2025-02-28,2024-09-17,2024-04-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
991,992.0,NUOVA A.S.EC. SOCIETA' COOPERATIVA,12949390152,20068.0,9.285357,45.441129,NaN,Active (default of payments),Cooperative company with limited liability by ...,Limited company,1999-10-25,10.0,2015-12-31,State of insolvency,Compulsory administrative liquidation,NaN,NaN,NaN,NaN,NaN,2019-08-01,2017-10-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
992,993.0,GESTIONE SERVIZI BOLOGNA SOCIETA' COOPERATIVA,3090921200,40050.0,11.349318,44.642602,NaN,Dissolved (bankruptcy),Cooperative company with limited liability by ...,Limited company,2011-01-17,4.0,2014-12-31,Conclusion of bankruptcy procedures,Bankruptcy,NaN,NaN,NaN,NaN,NaN,2019-10-21,2017-06-05,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
993,994.0,R.B. AUTOMAZIONI S.R.L.,2361240241,36050.0,11.369066,45.489839,NaN,Dissolved (liquidation),Limited liability company - SRL,Limited company,1993-09-22,10.0,2014-12-31,Conclusion of liquidation,Composition with creditors,Winding up and liquidation,NaN,NaN,NaN,NaN,2022-01-28,2015-09-02,2015-04-10,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
994,995.0,PULIZIA 360 S.R.L.,12643431005,141.0,NaN,NaN,NaN,Active,One-person company with limited liability - SRL,Limited company,